# Part I : Data collection using web scraping

Data scraping is the process of extracting information from a website and putting it into a spreadsheet. For a data collection activity, this method is an effective way to obtain a large amount of information for analysis, processing or inference. 

In our case, we want to create a recent dataset of phishing sites and legitimate sites. Therefore, we chose the phishtank.org site as a reference for phishing sites and we chose the most 1000 visited sites in 2021 as a reference for legitimate sites assuming that these sites are not phishing sites. Moreover, we extract the other part of ligitimate sites ( another 2000 links ) from an existing kaggle dataset that we merge into the other links to have finally a multisource data. 


## I.1 Phishing websites extraction using selenium

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import unittest
import csv
import re
import requests
from urllib.parse import urlparse,urlencode
import ipaddress
import socket
from bs4 import BeautifulSoup as bs

In [9]:
PATH = "./chromedriver.exe"
# chromedriver has to be installled in the same folder as the notebook in this case or you have to add the path 
driver = webdriver.Chrome(PATH)   
url_to_fetch="https://phishtank.org/"
driver.get(url_to_fetch)
#Naviger vers le site phishtank.org
print(driver.title) 



C:\Users\souhm\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until


PhishTank | Join the fight against phishing


In [10]:
#Go to search page
link=driver.find_element(By.LINK_TEXT,"Phish Search")
link.click()

In [11]:
#Search in website for urls that are confirmed to be phish
select = Select(driver.find_element(By.NAME,"valid"))
select.select_by_value('y')
button = driver.find_element(By.NAME,"Search")
button.click()

In [12]:
#extract XML data using XPATH : 

#nombre de lignes 
nbre_rows=len(driver.find_elements(By.XPATH,"//table/tbody/tr"))
print ("Number of rows", nbre_rows)
#nombre de colonnes
nbre_cols = len (driver.find_elements(By.XPATH,"//table/tbody/tr[1]/th"))
print("Number of columns"  ,nbre_cols)

Number of rows 22
Number of columns 5


In [13]:
"""
Here, I extract 20 websites per iteration and store them in a csv file that will be saved in the same folder 
K is the total number of iterations that we can fix according to the data size
Finally, we extract a total number of 20*K links
This process may take a while for large numbers of K

"""
def get_links_of_current_page(K):
    
    for i in range(K):
        for r in range(2, (nbre_rows)) :
            phish_website = []
            row=[]
            for c in [1,2,4,5]:
                x = driver.find_element(By.XPATH,"//tr["+str(r)+"]/td["+str(c)+"]").text
                row.append(x)
            phish_website.append(row[1:3])
            with open('phish_website.csv','a') as file: 
                file.writer=csv.writer(file)
                file.writer.writerow((phish_website[0][0],phish_website[0][1]))
            next = driver.find_element(By.XPATH,"//*[@id='widecol']/div/table/tbody/tr[22]/td[5]/b/a") # click older in phishtank
            next.click()

        file.close()
    
        #print(i/K) # to track the execution state
     



In [9]:
# This cell is used to run the code and extract 20000 phishing URL
get_links_of_current_page(1000)

PS: this code was inspired by the tutoriels of [**Tech with Tim youtube channel** ](https://www.youtube.com/watch?v=Xjv1sY630Uc&list=PLzMcBGfZo4-n40rB1XaJ0ak1bemvlqumQ&ab_channel=TechWithTim) 

## I.2 Kaggle dataset extraction

In [ ]:
import pandas as pd

data = pd.read_csv('./Datasets/dataset_phishing.csv')
data = data[ data["status"] == "legitimate"]

### Merging the partial datasets

In [ ]:
import csv

websites = list(data["url"]) + websites
with open('legitimate_websites', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    for i in websites:
      write.writerow([i])